# NcML creation

## Tools for accessing and processing climate data: Case study with R

****
![c4R](shared/figs/climate4R_esquema.png)

------------

This worked example contains the code that reproduces part of the examples shown in the paper ["climate4R: An R-based Framework for Climate Data Access, Post-processing and Bias Correction"](https://www.sciencedirect.com/science/article/pii/S1364815218303049).

In [1]:
library(loadeR)

Loading required package: rJava

Loading required package: loadeR.java

Java version 11x amd64 by Oracle Corporation detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.3 (2021-07-05) is loaded


Get the latest stable version (0.2.4) using <devtools::install_github('SantanderMetGroup/climate4R.UDG')>

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.7.1 (2021-07-05) is loaded


Get the latest stable version (1.8.1) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.



***
# 1. NcML creation

## 1.1. Observational reference

In [2]:
data.dir.obs <- "../../IMPETUS4CHANGE/data/BSC/CERRA/daily_mean/"

In [3]:
lf.obs <- list.files(data.dir.obs, pattern = "tas_", recursive = T)
lf.obs %>% head

[1] "tas_f3h-r2631x1113/tas_198409.nc" "tas_f3h-r2631x1113/tas_198410.nc"
[3] "tas_f3h-r2631x1113/tas_198411.nc" "tas_f3h-r2631x1113/tas_198412.nc"
[5] "tas_f3h-r2631x1113/tas_198501.nc" "tas_f3h-r2631x1113/tas_198502.nc"

In [4]:
dir.obs <- dirname(lf.obs) %>% unique
dir.obs

[1] "tas_f3h-r2631x1113"

In [5]:
ncml.dir.obs <- "../data/ncml/CERRA/"
dir.create(ncml.dir.obs, recursive = TRUE)

Warning message in dir.create(ncml.dir.obs, recursive = TRUE):
“'../data/ncml/CERRA' already exists”


In [6]:
makeAggregatedDataset(source.dir = paste0(data.dir.obs, dir.obs), 
                      ncml.file = paste0(ncml.dir.obs, dir.obs, ".ncml"),
                      aggr.dim = "time") %>% 
suppressMessages

In [7]:
dataset.obs <- list.files(ncml.dir.obs, full.names = T) 
dataset.obs

[1] "../data/ncml/CERRA//tas_f3h-r2631x1113.ncml"

In [8]:
di <- dataInventory(dataset.obs)

[2024-04-25 19:09:52] Doing inventory ...

[2024-04-25 19:09:53] Retrieving info for 'tas' (0 vars remaining)

[2024-04-25 19:09:53] Done.



In [9]:
di$tas$Dimensions$time$Date_range

[1] "1984-09-01T10:30:00Z - 2021-06-30T10:30:00Z"

## 1.2. Decadal predictions

In [10]:
data.dir <- "../../IMPETUS4CHANGE/data/ESGF/CMIP6/DCPP/EC-Earth-Consortium/EC-Earth3/dcppA-hindcast/"

In [11]:
lf <- list.files(data.dir, recursive = T, pattern = 'tas_.*hindcast')
lf %>% head; lf %>% tail

[1] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19601101-19611031.nc"
[2] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19611101-19621031.nc"
[3] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19621101-19631031.nc"
[4] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19631101-19641031.nc"
[5] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19641101-19651031.nc"
[6] "s1960-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s1960-r1i4p1f1_gr_19651101-19661031.nc"

[1] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20081101-20091031.nc"
[2] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20091101-20101031.nc"
[3] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20101101-20111031.nc"
[4] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20111101-20121031.nc"
[5] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20121101-20131031.nc"
[6] "s2003-r1i4p1f1/day/tas/gr/v20220322/tas_day_EC-Earth3_dcppA-hindcast_s2003-r1i4p1f1_gr_20131101-20141031.nc"

In [12]:
dir.inits <- dirname(lf) %>% unique
dir.inits %>% head; dir.inits %>% tail

[1] "s1960-r1i4p1f1/day/tas/gr/v20220322" "s1961-r1i4p1f1/day/tas/gr/v20220322"
[3] "s1962-r1i4p1f1/day/tas/gr/v20220322" "s1963-r1i4p1f1/day/tas/gr/v20220322"
[5] "s1964-r1i4p1f1/day/tas/gr/v20220322" "s1965-r1i4p1f1/day/tas/gr/v20220322"

[1] "s1998-r1i4p1f1/day/tas/gr/v20220322" "s1999-r1i4p1f1/day/tas/gr/v20220322"
[3] "s2000-r1i4p1f1/day/tas/gr/v20220322" "s2001-r1i4p1f1/day/tas/gr/v20220322"
[5] "s2002-r1i4p1f1/day/tas/gr/v20220322" "s2003-r1i4p1f1/day/tas/gr/v20220322"

In [13]:
ncml.dir <- "../data/ncml/EC-Earth3/dcppA-hindcast/"
dir.create(ncml.dir, recursive = TRUE)

Warning message in dir.create(ncml.dir, recursive = TRUE):
“'../data/ncml/EC-Earth3/dcppA-hindcast' already exists”


In [ ]:
lapply(dir.inits, function(x) 
                    makeAggregatedDataset(source.dir = paste0(data.dir, x), 
                                          ncml.file = paste0(ncml.dir, gsub("/", "_", x), ".ncml"),
                                          aggr.dim = "time")) %>% 
                    suppressMessages

In [15]:
datasets <- list.files(ncml.dir, full.names = T) 
datasets %>% head

[1] "../data/ncml/EC-Earth3/dcppA-hindcast//s1960-r1i4p1f1_day_tas_gr_v20220322.ncml"
[2] "../data/ncml/EC-Earth3/dcppA-hindcast//s1961-r1i4p1f1_day_tas_gr_v20220322.ncml"
[3] "../data/ncml/EC-Earth3/dcppA-hindcast//s1962-r1i4p1f1_day_tas_gr_v20220322.ncml"
[4] "../data/ncml/EC-Earth3/dcppA-hindcast//s1963-r1i4p1f1_day_tas_gr_v20220322.ncml"
[5] "../data/ncml/EC-Earth3/dcppA-hindcast//s1964-r1i4p1f1_day_tas_gr_v20220322.ncml"
[6] "../data/ncml/EC-Earth3/dcppA-hindcast//s1965-r1i4p1f1_day_tas_gr_v20220322.ncml"

### 1.3. Perform a data inventory

In [2]:
di <- dataInventory("../data/ncml/EC-Earth3/dcppA-hindcast//s1960-r1i4p1f1_day_tas_gr_v20220322.ncml")

[2024-04-26 08:39:08] Doing inventory ...

[2024-04-26 08:39:11] Retrieving info for 'tas' (0 vars remaining)

[2024-04-26 08:39:11] Done.



In [3]:
str(di)

List of 1
 $ tas:List of 7
  ..$ Description: chr "Near-Surface Air Temperature"
  ..$ DataType   : chr "float"
  ..$ Shape      : int [1:3] 4017 256 512
  ..$ Units      : chr "K"
  ..$ DataSizeMb : num 2106
  ..$ Version    : logi NA
  ..$ Dimensions :List of 3
  .. ..$ time:List of 4
  .. .. ..$ Type      : chr "Time"
  .. .. ..$ TimeStep  : chr "1.0 days"
  .. .. ..$ Units     : chr "days since 1850-01-01 00:00:00"
  .. .. ..$ Date_range: chr "1960-11-01T12:00:00Z - 1971-10-31T12:00:00Z"
  .. ..$ lat :List of 5
  .. .. ..$ Type       : chr "Lat"
  .. .. ..$ Units      : chr "degrees_north"
  .. .. ..$ Values     : num [1:256] -89.5 -88.8 -88.1 -87.4 -86.7 ...
  .. .. ..$ Shape      : int 256
  .. .. ..$ Coordinates: chr "lat"
  .. ..$ lon :List of 5
  .. .. ..$ Type       : chr "Lon"
  .. .. ..$ Units      : chr "degrees_east"
  .. .. ..$ Values     : num [1:512] 0 0.703 1.406 2.109 2.812 ...
  .. .. ..$ Shape      : int 512
  .. .. ..$ Coordinates: chr "lon"
